In [5]:
# full_trancate : 'customer_joining_info' Excel file from Drive to 'wh_online_banking' database

from urllib.parse import quote
import sqlalchemy as sqa
import pandas as pd
import pyodbc
import pymysql

# Step 2: Access Secondary/WH database
username = 'root'
password = quote('Bela@112')
host = 'localhost'
port = '3306'
database = 'wh_online_banking'

url = 'mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(username, password, host, port, database)
mysql_engine_2 = sqa.create_engine(url)

# Step 3: Load data from a CSV file
csv_file_path = r"D:\Farhan's Academi\Project Online Banking\customer_joining_info.csv"  

# Load CSV data into a DataFrame
df_csv = pd.read_csv(csv_file_path)

print(f"Number of rows loaded: {len(df_csv)}")

# Preview the CSV data
print("CSV Data:")
print(df_csv.head())

# Step 4: Optional - Load the CSV data into the Secondary/WH database 
df_csv.to_sql(
    name='customer_joining_info', 
    con=mysql_engine_2,
    if_exists='replace', 
    index=False,
    chunksize=1000 
)

print("CSV data has been loaded into the secondary database.")

Number of rows loaded: 3500
CSV Data:
   customer_id  region_id  area_id   join_date
0            1          3        4  2020-01-02
1            2          3        5  2020-01-03
2            3          5        4  2020-01-27
3            4          5        4  2020-01-07
4            5          3        3  2020-01-15
CSV data has been loaded into the secondary database.


In [15]:
# First 'region' is imported from drive to  'live_online_banking' database then full_trancate ↓
# full_trancate : 'region' dataset from'live_online_banking'to 'wh_online_banking' database

from urllib.parse import quote
import sqlalchemy as sqa
import pandas as pd
import pyodbc
import pymysql

## TO take the access of Primary / 'live_online_banking' database to pull the data table.

username = 'root'
password = quote('Bela@112')
host = 'localhost'
port = '3306'
database = 'live_online_banking'

url = 'mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(username, password, host, port, database)
mysql_engine = sqa.create_engine(url)
mysql_engine

## To take the access of secondary 'wh_online_banking' database to insert data table

username = 'root'
password = quote('Bela@112')
host = 'localhost'
port = '3306'
database = 'wh_online_banking'

url = 'mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(username, password, host, port, database)
mysql_engine_2 = sqa.create_engine(url)
mysql_engine_2

## Read SQL query from the 'mysql_engine' in LIVE database.
df1 = pd.read_sql_query("""select * from region;""", mysql_engine)

## to move the 'area' data table into the 'wh_online_banking' data base

df1.to_sql(
        name = 'region',
        con = mysql_engine_2,
        if_exists = 'replace',
        index = False,
        method = 'multi',
        chunksize = 1000
)

5

In [14]:
# First  'Area' is imported from drive to  'live_online_banking' database then full_trancate ↓
# full_trancate : 'Area' dataset from'live_online_banking'to 'wh_online_banking' database

from urllib.parse import quote
import sqlalchemy as sqa
import pandas as pd
import pyodbc
import pymysql

## TO take the access of Primary / 'live_online_banking' database to pull the data table.

username = 'root'
password = quote('Bela@112')
host = 'localhost'
port = '3306'
database = 'live_online_banking'

url = 'mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(username, password, host, port, database)
mysql_engine = sqa.create_engine(url)
mysql_engine

## To take the access of secondary 'wh_online_banking' database to insert data table

username = 'root'
password = quote('Bela@112')
host = 'localhost'
port = '3306'
database = 'wh_online_banking'

url = 'mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(username, password, host, port, database)
mysql_engine_2 = sqa.create_engine(url)
mysql_engine_2

## Read SQL query from the 'mysql_engine' in LIVE database.
df1 = pd.read_sql_query("""select * from area;""", mysql_engine)

## to move the 'area' data table into the 'wh_online_banking' data base

df1.to_sql(
        name = 'area',
        con = mysql_engine_2,
        if_exists = 'replace',
        index = False,
        method = 'multi',
        chunksize = 1000
)

5

In [18]:
# First 'customer_transactions' is imported from drive to  'live_online_banking' database then Incremental Load ↓

## Incremental Load : 'customer_transactions' dataset from 'live_online_banking' to 'wh_online_banking' database

# # Import necessary libraries for database connection and data manipulation
from urllib.parse import quote
import sqlalchemy as sqa
import pandas as pd
import pyodbc
import pymysql
import mysql.connector

# # Step 1: Define connection parameters for the 'live_online_banking' database
username = 'root'
password = quote('Bela@112')
host = 'localhost'
port = '3306'
database = 'live_online_banking'

# Step 2: Create an SQLAlchemy engine for the 'live_online_banking' database
url = 'mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(username, password, host, port, database)
mysql_engine = sqa.create_engine(url)
mysql_engine

# Step 3: Define connection parameters for the 'wh_online_banking' database (data warehouse)
username = 'root'
password = quote('Bela@112')
host = 'localhost'
port = '3306'
database = 'wh_online_banking'

# Step 4: Create an SQLAlchemy engine for the 'wh_online_banking' database
url = 'mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(username, password, host, port, database)
mysql_engine_2 = sqa.create_engine(url)
mysql_engine_2

# Step 5: Perform an incremental load query to fetch only new data
df = pd.read_sql_query("""select *
        from customer_transactions
            where customer_transactions.customer_id > (select  coalesce((select id from  store_customer_id),0));""",mysql_engine)

print(df)

# Step 6: Append the fetched data into the 'customer_transactions' table in the target database
df.to_sql(
        name = 'customer_transactions',
        con = mysql_engine_2,
        if_exists = 'append',
        index = False,
        method = 'multi',
        chunksize = 1000
)

# Step 7: Truncate the 'store_customer_id' table to prepare for the next max ID storage
with mysql_engine.connect() as connection:
    connection.execute(sqa.text('TRUNCATE TABLE  store_customer_id;'))


## Step 8: Fetch the maximum customer_id from the source 'customer_transactions' table
select_max_query = """SELECT MAX(customer_id) FROM customer_transactions;"""
with mysql_engine.connect() as connection:
    max_customer_id = connection.execute(sqa.text(select_max_query)).fetchone()
    print("customer_id:", max_customer_id)  

# Step 9: Insert the maximum customer_id into the 'store_customer_id' table
if max_order_id:
    insert_query = """INSERT INTO store_customer_id
                      SELECT :max_customer_id;""" 

    with mysql_engine.connect() as connection:
        connection.execute(sqa.text(insert_query), {'max_customer_id': max_order_id[0]})
        connection.commit()  

        print(f"Inserted max_customer_id: {max_order_id[0]} into store_customer_id.")
else:
    print("No customer_id found in customer_transactions.")




      customer_id    txn_date    txn_type  txn_amount
0             429  2020-01-21     deposit          82
1             155  2020-01-10     deposit         712
2             398  2020-01-01     deposit         196
3             255  2020-01-14     deposit         563
4             185  2020-01-29     deposit         626
...           ...         ...         ...         ...
5863          189  2020-02-03  withdrawal         870
5864          189  2020-03-22     deposit         718
5865          189  2020-02-06     deposit         393
5866          189  2020-01-22     deposit         302
5867          189  2020-01-27  withdrawal         861

[5868 rows x 4 columns]
customer_id: (500,)
Inserted max_customer_id: 500 into store_customer_id.
